In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
path = ('.../creditcard.csv')
df1 = pd.read_csv(path)

In [49]:
df1

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [5]:
# count of each class

print(df1[df1['Class']==1].shape[0])
print(df1[df1['Class']==0].shape[0])
print(df1[df1['Class']==1].shape[0]/df1.shape[0])

492
284315
0.001727485630620034


In [6]:
# rebalance   

df3 = df1[df1['Class']==1]

df4 = df1[df1['Class']==0].sample(frac=0.3)

In [7]:
df = pd.concat([df3,df4])

In [9]:
print(df[df['Class']==1].shape[0])
print(df[df['Class']==0].shape[0])
print(df[df['Class']==1].shape[0]/df.shape[0])

492
85294
0.0057352015480381414


In [10]:
df.drop_duplicates(inplace=True)

In [55]:
# correlations with target column

corr_rel = []
col_name = []
for i in df.columns:
    if i!='Class':
        corr_rel.append(df['Class'].corr(df[i]))
        col_name.append(i)
xxx = pd.DataFrame(zip(col_name,corr_rel),columns=['Column_name','Corr_effc'])

xxx['Corr_effc'] = abs(xxx['Corr_effc'])

xx = xxx.sort_values('Corr_effc',ascending = False)

xx.index = np.arange(0,xx.shape[0])
xx

,Column_name,Corr_effc
0,V14,0.474389
1,V17,0.466571
2,V12,0.411674
3,V10,0.347467
4,V16,0.316862
5,V3,0.307209
6,V7,0.279377
7,V11,0.260737
8,V4,0.229275
9,V18,0.185430


In [56]:
clf = xgb.XGBClassifier(random_state = 50)

In [57]:
# forward elimination from 7 features to 12 features

features = []
accur = []
f1_scor = []

for i in range(7,13):
    y = df['Class']
    x = df[xx.iloc[:i,0]]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state = 50)

    clf.fit(x_train, y_train)
    y_pred=clf.predict(x_test)

    features.append(i)
    accur.append(accuracy_score(y_test, y_pred))
    f1_scor.append(f1_score(y_test, y_pred))
    print('with', i,'feature')
    print('accuracy_score', round(accuracy_score(y_test, y_pred),4))
    print('f1_score', round(f1_score(y_test, y_pred),4))
    print('----------------------------------')

with 7 feature
accuracy_score 0.9986
f1_score 0.8689
----------------------------------
with 8 feature
accuracy_score 0.9987
f1_score 0.8773
----------------------------------
with 9 feature
accuracy_score 0.9988
f1_score 0.8806
----------------------------------
with 10 feature
accuracy_score 0.9989
f1_score 0.893
----------------------------------
with 11 feature
accuracy_score 0.9989
f1_score 0.893
----------------------------------
with 12 feature
accuracy_score 0.9988
f1_score 0.8889
----------------------------------


In [58]:
# define optimum features count

Feat_el = pd.DataFrame(zip(features,accur,f1_scor),columns=['Feature_count','Accuracy_scor','F1_scor'])
Feat_el.sort_values('F1_scor',ascending=False)

,Feature_count,Accuracy_scor,F1_scor
3,10,0.998871,0.892989
4,11,0.998871,0.892989
5,12,0.998832,0.888889
2,9,0.998754,0.880597
1,8,0.998716,0.877323
0,7,0.998638,0.868914


In [59]:
optimum_feat_count = Feat_el.sort_values('F1_scor',ascending=False).iloc[0,0]
optimum_feat_count

10

In [61]:
x = df[xx['Column_name'].values[:optimum_feat_count]]
y = df['Class']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state = 50)

In [62]:
# grid search

grid_param = {
    'learning_rate': [0.1, 0.2,0.3],
    'max_depth': range(2,7,1),
    'n_estimators': [90,100],
}

grid_search = GridSearchCV(estimator=clf, param_grid=grid_param)

grid_search.fit(x_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Corresponding mean test score: ", grid_search.best_score_)

best_params = grid_search.best_params_ 

Best parameters:  {'learning_rate': 0.2, 'max_depth': 2, 'n_estimators': 90}
Corresponding mean test score:  0.9987989337738075


In [63]:
# with best parametres

clf = xgb.XGBClassifier(**best_params)

clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)

print('accuracy_score', accuracy_score(y_test, y_pred))
print('f1_score', f1_score(y_test, y_pred))

accuracy_score 0.9987544761015102
f1_score 0.8814814814814814
